<a href="https://colab.research.google.com/github/yagnesh2004/Python/blob/main/medical_telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#AIMER Society - Indian Servers
!pip install pyTelegramBotAPI
!pip install openai
!pip install opencage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import telebot
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic

bot = telebot.TeleBot('6049992217:AAGnHYo1-_vYvKoqIkm0NixoBTrBjra0y34')

def get_nearby_emergency_services(latitude, longitude):
    # Search radius in meters
    radius = 10000

    # Get hospitals within the search radius
    hospitals = get_places_nearby(latitude, longitude, "hospital", radius)
    # Get clinics within the search radius
    clinics = get_places_nearby(latitude, longitude, "clinic", radius)
    # Get pharmacies within the search radius
    pharmacies = get_places_nearby(latitude, longitude, "pharmacy", radius)
    # Get ambulances within the search radius
    ambulances = get_places_nearby(latitude, longitude, "ambulance", radius)

    return {
        "Hospital": hospitals,
        "Clinic": clinics,
        "Pharmacy": pharmacies,
        "Ambulance": ambulances
    }

def get_places_nearby(latitude, longitude, keyword, radius):
    geocoder = OpenCageGeocode('71b7bcbe92e947e897df5e480be8a06a')
    results = geocoder.reverse_geocode(latitude, longitude)
    address = results[0]['formatted']

    places = []
    places_raw = geocoder.geocode(address, no_annotations=1, limit=10, countrycode='in', q=keyword)
    for place in places_raw:
        place_address = place['formatted']
        place_coords = (place['geometry']['lat'], place['geometry']['lng'])
        distance = calculate_distance((latitude, longitude), place_coords)
        places.append({"address": place_address, "distance": distance})

    places.sort(key=lambda x: x["distance"])

    return places

def calculate_distance(coords1, coords2):
    return round(geodesic(coords1, coords2).kilometers, 2)

@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "Welcome! Please share your location to find nearby emergency services.")

@bot.message_handler(content_types=['location'])
def handle_location(message):
    latitude = message.location.latitude
    longitude = message.location.longitude

    # Get nearby emergency services
    places = get_nearby_emergency_services(latitude, longitude)

    if places:
        response = "Nearby emergency services:\n\n"
        for category, places_list in places.items():
            if places_list:
                response += category + ":\n"
                for place in places_list:
                    response += "- {} (Distance: {} km)\n".format(place["address"], place["distance"])
                response += "\n"
        else:
            response = "No emergency services found nearby."
    else:
        response = "Error occurred while fetching emergency services."

    bot.reply_to(message, response)

bot.polling()
